# NLP - Prétraitement de Texte
## Cours NLP - Techniques essentielles de traitement de texte

Ce notebook couvre les techniques fondamentales du prétraitement de texte en NLP avec des fonctions réutilisables.

## 1. Chargement et Lecture du Texte

Fonction réutilisable pour charger des fichiers texte.

In [ ]:
import subprocess
import sys

packages = ['nltk', 'spacy', 'unidecode', 'numpy', 'pandas']
for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

print("✓ Bibliothèques installées avec succès")

In [ ]:
# Imports
import nltk
import string
import re
import unicodedata
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from unidecode import unidecode
import spacy
from pathlib import Path

# Téléchargement des ressources NLTK
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# Chargement du modèle spaCy français
try:
    nlp = spacy.load('fr_core_news_sm')
except OSError:
    print("Téléchargement du modèle spaCy français...")
    subprocess.check_call([sys.executable, '-m', 'spacy', 'download', 'fr_core_news_sm'])
    nlp = spacy.load('fr_core_news_sm')

print("✓ Imports et modèles chargés")

In [ ]:
def load_text_file(file_path):
    """
    Charge un fichier texte et retourne son contenu.
    
    Args:
        file_path (str): Chemin vers le fichier texte
    
    Returns:
        str: Contenu du fichier
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        return text
    except FileNotFoundError:
        print(f"Erreur: Le fichier {file_path} n'existe pas")
        return ""
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier: {e}")
        return ""

# Charger le texte d'entrée
text = load_text_file('entree.txt')
print(f"Texte chargé: {len(text)} caractères")
print("\n--- Premier 200 caractères ---")
print(text[:200] + "...")

## 2. Tokenisation en Mots

Fonction pour tokeniser le texte en mots individuels.

In [ ]:
def tokenize_words(text):
    """
    Tokenise un texte en mots individuels.
    
    Args:
        text (str): Texte à tokeniser
    
    Returns:
        list: Liste des mots
    """
    tokens = word_tokenize(text, language='french')
    return tokens

# Tester la tokenisation en mots
words = tokenize_words(text)
print(f"Nombre de mots: {len(words)}")
print(f"Premiers 20 mots: {words[:20]}")

## 3. Tokenisation en Phrases

Fonction pour tokeniser le texte en phrases.

In [ ]:
def tokenize_sentences(text):
    """
    Tokenise un texte en phrases.
    
    Args:
        text (str): Texte à tokeniser
    
    Returns:
        list: Liste des phrases
    """
    sentences = sent_tokenize(text, language='french')
    return sentences

# Tester la tokenisation en phrases
sentences = tokenize_sentences(text)
print(f"Nombre de phrases: {len(sentences)}")
print("\n--- Premières 3 phrases ---")
for i, sent in enumerate(sentences[:3], 1):
    print(f"{i}. {sent}")

## 4. Génération de Bigrammes

Fonction pour transformer une liste de mots en bigrammes (paires de mots consécutifs).

In [ ]:
def generate_bigrams(words):
    """
    Génère des bigrammes à partir d'une liste de mots.
    Un bigramme est une paire de deux mots consécutifs.
    
    Args:
        words (list): Liste des mots
    
    Returns:
        list: Liste des bigrammes (tuples de 2 mots)
    """
    bigrams = [(words[i], words[i+1]) for i in range(len(words)-1)]
    return bigrams

# Tester la génération de bigrammes
sample_words = words[:10]
bigrams = generate_bigrams(sample_words)
print(f"Mots: {sample_words}")
print(f"\nBigrammes générés: {bigrams}")
print(f"\nNombre total de bigrammes dans le texte: {len(generate_bigrams(words))}")

## 5. Normalisation Unicode

Fonction complète de normalisation: minuscules, suppression accents, normalisation des nombres.

In [ ]:
def normalize_text(text):
    """
    Normalise le texte:
    - Conversion en minuscules
    - Suppression des accents
    - Normalisation des nombres
    
    Args:
        text (str): Texte à normaliser
    
    Returns:
        str: Texte normalisé
    """
    # Convertir en minuscules
    text = text.lower()
    
    # Supprimer les accents avec unidecode
    text = unidecode(text)
    
    # Normalisation des nombres (optionnel: remplacer par <NUM>)
    # Décommentez la ligne suivante pour remplacer les nombres par <NUM>
    # text = re.sub(r'\d+', '<NUM>', text)
    
    return text

# Tester la normalisation
sample_text = "Noël est le 25 décembre. C'est une fêtE avec Événements !"
normalized = normalize_text(sample_text)
print(f"Original: {sample_text}")
print(f"Normalisé: {normalized}")

# Appliquer sur le texte complet
normalized_text = normalize_text(text)
print(f"\n✓ Texte entier normalisé ({len(normalized_text)} caractères)")

## 6. Suppression de Ponctuation et Stop Words

Fonction pour supprimer la ponctuation et les mots vides (stop words).

In [ ]:
# Charger les stop words français
french_stopwords = set(stopwords.words('french'))

def remove_punctuation_and_stopwords(words, remove_stopwords=True):
    """
    Supprime la ponctuation et optionnellement les stop words.
    
    Args:
        words (list): Liste des mots (tokens)
        remove_stopwords (bool): Si True, supprime les stop words
    
    Returns:
        list: Liste des mots filtrés
    """
    # Supprimer la ponctuation
    filtered = [word for word in words if word not in string.punctuation]
    
    # Supprimer les stop words
    if remove_stopwords:
        filtered = [word for word in filtered if word.lower() not in french_stopwords]
    
    return filtered

# Tester sur les premiers mots
sample_words_raw = tokenize_words("Bonjour, ceci est un test ! Les stop words doivent être supprimés.")
print(f"Avant filtrage: {sample_words_raw}")
filtered_words = remove_punctuation_and_stopwords(sample_words_raw)
print(f"\nAprès filtrage: {filtered_words}")

# Appliquer sur le texte complet
normalized_and_tokenized = tokenize_words(normalized_text)
clean_words = remove_punctuation_and_stopwords(normalized_and_tokenized)
print(f"\nTexte complet:")
print(f"Mots avec stop words et ponctuation: {len(normalized_and_tokenized)}")
print(f"Mots filtrés: {len(clean_words)}")
print(f"Premiers mots filtrés: {clean_words[:15]}")

## 7. Racinisation (Stemming)

Fonction de stemming pour réduire les mots à leur racine.

In [ ]:
# Initialiser le stemmer français (Snowball)
stemmer = SnowballStemmer('french')

def stem_words(words):
    """
    Applique le stemming (racinisation) à une liste de mots.
    Réduit les mots à leur racine/radical.
    
    Args:
        words (list): Liste des mots
    
    Returns:
        list: Liste des mots racinisés
    """
    stemmed = [stemmer.stem(word) for word in words]
    return stemmed

# Tester le stemming
test_words = ['courant', 'courants', 'courant', 'couru', 'courserait', 
              'clair', 'clarté', 'clairement', 'briller', 'brillant']
stemmed = stem_words(test_words)

print("Stemming - Réduction à la racine:")
print("-" * 50)
for original, stem in zip(test_words, stemmed):
    print(f"{original:15} → {stem}")

# Appliquer sur les mots nettoyés
stemmed_words = stem_words(clean_words)
print(f"\n✓ {len(stemmed_words)} mots racinisés")
print(f"Exemple: {clean_words[:5]} → {stemmed_words[:5]}")

## 8. Lemmatisation

Fonction de lemmatisation pour ramener les mots à leur forme canonique.

In [ ]:
def lemmatize_text(text):
    """
    Applique la lemmatisation au texte en utilisant spaCy.
    Ramène chaque mot à sa forme canonique (lemme).
    
    Args:
        text (str): Texte à lemmatiser
    
    Returns:
        list: Liste des lemmes
    """
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return lemmas

# Tester la lemmatisation
test_text = "courant courants couru courserait clair clarté clairement briller brillant"
lemmas = lemmatize_text(test_text)

print("Lemmatisation avec spaCy:")
print("-" * 50)
doc = nlp(test_text)
for token in doc:
    print(f"{token.text:15} → {token.lemma_:15} ({token.pos_})")

# Appliquer sur le texte complet
lemmas_full = lemmatize_text(normalized_text)
print(f"\n✓ {len(lemmas_full)} lemmes générés")
print(f"Premiers lemmes: {lemmas_full[:15]}")

## 9. Pipeline Complet de Prétraitement

Exemple d'utilisation du pipeline complet.

In [ ]:
class TextPreprocessor:
    """
    Classe réutilisable pour le prétraitement de texte (style Java).
    Permet de chaîner les opérations de nettoyage et de traitement.
    """
    
    def __init__(self):
        self.original_text = None
        self.processed_text = None
        self.tokens = None
        self.sentences = None
    
    def load_file(self, file_path):
        """Charge un fichier texte"""
        self.original_text = load_text_file(file_path)
        return self
    
    def set_text(self, text):
        """Définit le texte directement"""
        self.original_text = text
        self.processed_text = text
        return self
    
    def normalize(self):
        """Applique la normalisation Unicode"""
        if self.processed_text is None:
            self.processed_text = self.original_text
        self.processed_text = normalize_text(self.processed_text)
        return self
    
    def tokenize_into_words(self):
        """Tokenise en mots"""
        text = self.processed_text if self.processed_text else self.original_text
        self.tokens = tokenize_words(text)
        return self
    
    def tokenize_into_sentences(self):
        """Tokenise en phrases"""
        text = self.original_text if self.original_text else self.processed_text
        self.sentences = tokenize_sentences(text)
        return self
    
    def clean_tokens(self, remove_stopwords=True):
        """Supprime ponctuation et stop words"""
        if self.tokens is None:
            self.tokenize_into_words()
        self.tokens = remove_punctuation_and_stopwords(self.tokens, remove_stopwords)
        return self
    
    def stem(self):
        """Applique le stemming"""
        if self.tokens is None:
            self.tokenize_into_words()
        self.tokens = stem_words(self.tokens)
        return self
    
    def lemmatize(self):
        """Applique la lemmatisation"""
        text = self.processed_text if self.processed_text else self.original_text
        lemmas = lemmatize_text(text)
        self.tokens = lemmas
        return self
    
    def get_bigrams(self):
        """Génère les bigrammes"""
        if self.tokens is None:
            self.tokenize_into_words()
        return generate_bigrams(self.tokens)
    
    def get_tokens(self):
        """Retourne les tokens actuels"""
        return self.tokens
    
    def get_sentences(self):
        """Retourne les phrases"""
        return self.sentences
    
    def get_text(self):
        """Retourne le texte traité"""
        return self.processed_text if self.processed_text else self.original_text


# Tester la classe
print("=" * 60)
print("PIPELINE COMPLET - Style Java avec Builder Pattern")
print("=" * 60)

# Créer un préprocesseur et appliquer les transformations
processor = TextPreprocessor()
processor.load_file('entree.txt').normalize().tokenize_into_words().clean_tokens()

print(f"\n✓ Texte chargé, normalisé et tokenisé")
print(f"  Nombre de tokens: {len(processor.get_tokens())}")
print(f"  Premiers tokens: {processor.get_tokens()[:10]}")

# Générer des bigrammes
bigrams = processor.get_bigrams()
print(f"\n✓ Bigrammes générés: {len(bigrams)}")
print(f"  Exemples: {bigrams[:5]}")

# Appliquer le stemming
processor.stem()
print(f"\n✓ Stemming appliqué")
print(f"  Tokens après stemming: {processor.get_tokens()[:10]}")

## 10. Comparaison des Approches

Comparaison entre stemming et lemmatisation.

In [ ]:
import pandas as pd

# Texte d'exemple
example_text = "Les restaurants servent à manger des mets délicieux. Manger est une action naturelle pour les mangeurs."

print("COMPARAISON: STEMMING vs LEMMATISATION")
print("=" * 70)
print(f"Texte original: {example_text}\n")

# Tokenisation
words = tokenize_words(example_text)
normalized_words = normalize_text(example_text)
tokens = tokenize_words(normalized_words)

# Stemming
stemmed = stem_words(tokens)

# Lemmatisation
lemmas = lemmatize_text(example_text)

# Créer un tableau de comparaison
data = {
    'Mot Original': tokens,
    'Stemming': stemmed,
    'Lemmatisation': lemmas
}

df = pd.DataFrame(data)
print(df.to_string(index=False))

print("\n" + "=" * 70)
print("OBSERVATIONS:")
print("-" * 70)
print("• STEMMING: Coupe les mots de manière algorithmique (plus agressif)")
print("  Exemple: 'mangeurs' → 'mang'")
print("\n• LEMMATISATION: Ramène à la forme de base grammaticale (plus linguistique)")
print("  Exemple: 'mangeurs' → 'mangeur'")
print("\nSTEMMING est plus rapide mais peut être trop agressif")
print("LEMMATISATION est plus précis mais requiert plus de ressources")

## 11. Résumé des Fonctions

Guide d'utilisation des fonctions principales.

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════════════════╗
║                    RÉSUMÉ DES FONCTIONS DISPONIBLES                        ║
╚════════════════════════════════════════════════════════════════════════════╝

1. CHARGEMENT DE TEXTE
   └─ load_text_file(file_path)
      Charge un fichier texte et retourne son contenu

2. TOKENISATION
   ├─ tokenize_words(text)
   │  Divise le texte en mots individuels
   └─ tokenize_sentences(text)
      Divise le texte en phrases

3. BIGRAMMES
   └─ generate_bigrams(words)
      Crée des paires de mots consécutifs

4. NORMALISATION UNICODE
   └─ normalize_text(text)
      - Convertit en minuscules
      - Supprime les accents
      - Normalise les nombres

5. NETTOYAGE
   └─ remove_punctuation_and_stopwords(words, remove_stopwords=True)
      Supprime la ponctuation et les mots vides

6. RACINISATION
   └─ stem_words(words)
      Réduit les mots à leur racine (algorithme Snowball)

7. LEMMATISATION
   └─ lemmatize_text(text)
      Ramène les mots à leur forme canonique (spaCy)

8. CLASSE RÉUTILISABLE
   └─ TextPreprocessor
      Classe avec méthodes chaînables (Builder Pattern):
      - load_file(path)
      - set_text(text)
      - normalize()
      - tokenize_into_words()
      - tokenize_into_sentences()
      - clean_tokens(remove_stopwords=True)
      - stem()
      - lemmatize()
      - get_bigrams()
      - get_tokens()
      - get_sentences()
      - get_text()

╔════════════════════════════════════════════════════════════════════════════╗
║                         EXEMPLES D'UTILISATION                             ║
╚════════════════════════════════════════════════════════════════════════════╝
""")

# EXEMPLE 1: Utilisation simple
print("\\n[EXEMPLE 1] - Utilisation simple")
print("-" * 80)
text1 = "Bonjour, ceci est un texte d'exemple avec des accents: café, élève, français !"
print(f"Original: {text1}")
print(f"Normalisé: {normalize_text(text1)}")

# EXEMPLE 2: Pipeline complet avec la classe
print("\\n[EXEMPLE 2] - Pipeline complet avec TextPreprocessor")
print("-" * 80)
processor2 = TextPreprocessor()
result2 = (processor2
    .set_text("Les enfants jouent dans le parc. Jouer est amusant!")
    .normalize()
    .tokenize_into_words()
    .clean_tokens()
)
print(f"Tokens filtrés: {result2.get_tokens()}")

# EXEMPLE 3: Comparaison stemming vs lemmatisation
print("\\n[EXEMPLE 3] - Stemming vs Lemmatisation")
print("-" * 80)
test_text3 = "marcher marcheurs marche marchant"
words3 = tokenize_words(test_text3)
print(f"Original:       {words3}")
print(f"Stemming:       {stem_words(words3)}")
print(f"Lemmatisation:  {lemmatize_text(test_text3)}")

# EXEMPLE 4: Bigrammes
print("\\n[EXEMPLE 4] - Génération de bigrammes")
print("-" * 80)
text4 = "Python est un langage de programmation"
words4 = tokenize_words(normalize_text(text4))
bigrams4 = generate_bigrams(words4)
print(f"Mots: {words4}")
print(f"Bigrammes: {bigrams4}")

print("\\n" + "=" * 80)
print("✓ Notebook NLP complet prêt à l'utilisation!")
print("=" * 80)